# 🏋️ Powerlifting SQL Analysis

In this notebook, we explore a powerlifting dataset using SQL. Each query answers a specific question related to performance, ranking, and trends.

### 1. Average Amount Lifted by Lift Type

In [ ]:
SELECT 
    `Lift Type`,
    AVG(`Amount Lifted (kg)`) AS avg_lift
FROM 
    powerlifting_dataset
GROUP BY 
    `Lift Type`;

### 2. Heaviest Lift Per Lifter

In [ ]:
SELECT 
    `Lifter Name`, 
    MAX(`Amount Lifted (kg)`) AS max_amount_lifted
FROM 
    powerlifting_dataset
GROUP BY 
    `Lifter Name`;

### 3. Lifter with the Highest Total Amount Lifted

In [ ]:
SELECT 
    `Lifter Name`, 
    SUM(`Amount Lifted (kg)`) AS total_amount_lifted
FROM 
    powerlifting_dataset
GROUP BY 
    `Lifter Name`
ORDER BY 
    total_amount_lifted DESC
LIMIT 1;

### 4. Total Lifts Per Weight Class

In [ ]:
SELECT 
    COUNT(`Amount Lifted (kg)`) AS total_lifted, 
    `Weight Class`
FROM 
    powerlifting_dataset
GROUP BY 
    `Weight Class`;

### 5. Lifters Who Performed More Than One Lift Type

In [ ]:
SELECT 
    `Lifter Name`
FROM 
    powerlifting_dataset
GROUP BY 
    `Lifter Name`
HAVING 
    COUNT(DISTINCT `Lift Type`) > 1;

### 6. Lifters Who Improved Over Time

In [ ]:
WITH pf AS (
    SELECT 
        `Lifter Name`, 
        `Age`, 
        `Lift Type`, 
        `Amount Lifted (kg)` AS amount, 
        LAG(`Amount Lifted (kg)`) OVER (
            PARTITION BY `Lifter Name`, `Lift Type`
            ORDER BY `Age`
        ) AS previous_lift
    FROM 
        powerlifting_dataset
) 
SELECT * 
FROM pf 
WHERE 
    amount > previous_lift;

### 7. Top Lifter by Lift Type

In [ ]:
WITH rank_lift AS (
    SELECT 
        `Lifter Name`, 
        `Weight Class`, 
        `Lift Type`, 
        `Amount Lifted (kg)` AS Amount, 
        ROW_NUMBER () OVER (
            PARTITION BY `Lift Type` 
            ORDER BY `Amount Lifted (kg)` DESC 
        ) AS row_nm
    FROM 
        powerlifting_dataset
) 
SELECT 
    `Lifter Name`, 
    `Weight Class`, 
    `Lift Type`,
    Amount
FROM rank_lift
WHERE row_nm = 1;

### 8. Average Lifted by Age Group

In [ ]:
SELECT 
    CONCAT(FLOOR(Age / 10) * 10, '-' ,FLOOR(Age / 10) * 10 + 9) AS age_group, 
    ROUND(AVG(`Amount Lifted (kg)`)) AS avg_lifted
FROM 
    powerlifting_dataset
GROUP BY
    age_group
ORDER BY 
    age_group;

### 9. Normalize Weight Class and Explore Correlation

In [ ]:
SELECT 
    REPLACE(`Weight Class`, 'kg', '') AS weight_class_kg, 
    `Amount Lifted (kg)`
FROM 
    powerlifting_dataset
WHERE 
    `Weight Class` IS NOT NULL
    AND `Amount Lifted (kg)` IS NOT NULL;

### 10. Global Lift Ranking

In [ ]:
SELECT 
    `Lifter Name`, 
    `Lift Type`, 
    `Amount Lifted (kg)`, 
    RANK () OVER (
        ORDER BY `Amount Lifted (kg)` DESC
    ) AS `rank` 
FROM 
    powerlifting_dataset;